 # About
 This notebook refines the resampled BKGN files based on standard deviation and also saves the tiff files as images for quick visualization

 # Libraries

In [4]:
from tqdm.notebook import tqdm as td 
import rasterio as rio, matplotlib.pyplot as plt, time, glob, numpy as np, shutil, os
from osgeo import gdal
def fresh(where):
    if os.path.exists(where):
        shutil.rmtree(where)
        os.mkdir(where)
    else:
        os.mkdir(where)  
start = time.time()

 # Read files and set vars

In [5]:
rasBKGN = sorted(glob.glob("../../3_RSData/1_Rasters/2_BKGN/03_resampled/*.tif"))  
dfBKGN = {}  # dict to store rasters
for i in range(len(rasBKGN)):
    dfBKGN[i] = rio.open(rasBKGN[i]).read(1)  # time slices read into a dict of numpy arrays

 # Filter data

In [6]:
for k in td((range(len(rasBKGN))), desc='Removing outliers'):  # 2 filter outliers
    data = dfBKGN[k].flatten('F')  # std and mean
    dataSD = np.std(data)
    dataMean = np.mean(data)
    lower = dataMean - (dataSD*3) # lower and upper limits of data within 3SDs
    upper = dataMean + (dataSD*3)
    outliers = []  # get outliers in a list
    for item in data:
        if item > upper or item < lower:
            outliers.append(item)
    row = dfBKGN[k].shape[0]  # remove outliers
    col = dfBKGN[k].shape[1]
    for i in range(0, 66):
        for j in range(0, 70):
            if dfBKGN[k][i,j] in outliers:
                dfBKGN[k][i,j] = np.nan  # replace the outlier entry by np.nan
                dfBKGN[k][i,j] = np.nanmean(dfBKGN[k][i-1:i+2, j-1:j+2])  # or 3x3 window avg
            else:
                pass

Removing outliers:   0%|          | 0/12 [00:00<?, ?it/s]

C:\Users\bcyadav\AppData\Local\Temp/ipykernel_8612/3139919152.py:17: RuntimeWarning: Mean of empty slice
  dfBKGN[k][i,j] = np.nanmean(dfBKGN[k][i-1:i+2, j-1:j+2])  # or 3x3 window avg


 # Save filtered data tiff files

In [4]:
referenceRas = gdal.Open(rasBKGN[0])  # reference raster
outDir = "../../3_RSData/1_Rasters/2_BKGN/04_refined/"
dsTiffBKGN = {}

for i in td(range(len(rasBKGN)), desc='Saving as tiff'):
    date = rasBKGN[i].split('/')[-1].split('.')[0]  # date of file extracted
    dsTiffBKGN[i] = gdal.GetDriverByName('GTiff').Create(
        outDir + "{}.tif".format(date), 
        xsize=referenceRas.RasterXSize, 
        ysize=referenceRas.RasterYSize, 
        bands=1, 
        eType=gdal.GDT_Float32)  
    dsTiffBKGN[i].SetGeoTransform(referenceRas.GetGeoTransform())  # geotransform set
    dsTiffBKGN[i].SetProjection(referenceRas.GetProjection())  # projection set (UTM44N)
    dsTiffBKGN[i].GetRasterBand(1).WriteArray(dfBKGN[i])  # data written to tiff file
    dsTiffBKGN[i]=None  # tiff file closed after creation

 # Also save filtered data images

## All images

In [5]:
outDir = "../../5_Images/00_BKGNData/02_refined/"  # 3 saving images of refined data
# fresh(outDir)  # old images removed
for i in td(range(len(rasBKGN)), desc='Saving images'):  
    plt.figure(dpi=300)  # plotting
    plt.imshow(dfBKGN[i], cmap='jet')
    cbar = plt.colorbar(extend='both')
    cbar.set_label('mm/month', rotation=90)
    plt.title("BKGN's Precipitation in {}".format(rasBKGN[i].split('/')[-1][3:6]))
    plt.xlabel('Grid columns')
    plt.ylabel('Grid rows')
    # plt.savefig(outDir + rasBKGN[i].split('\\')[-1].split('.')[0], bbox_inches='tight', facecolor='w')
    plt.close()

Saving images:   0%|          | 0/12 [00:00<?, ?it/s]

## Only JJAS

In [6]:
print('Time elapsed: ', np.round(time.time()-start,2), 'secs')

Time elapsed:  4.37 secs
